# Custom Dataset

This notebook shows how to create a custom dataset in PyTorch.
We give to the dataset a CSV file with the following columns:
- `index`: The unique identifier for each tweet.
- `tweet_text_clean`: The cleaned version of the tweet text. (no URLS and mentions)
- `img_text`: The text extracted from the image, if available. (otherwise NaN)
- `binary_hate`: A binary label indicating whether the tweet contains hate speech or not. (threshold at 0.5)
- `split`: The split of the dataset (train, test, or val).
And the folder containing the images (each should be named as `index.jpg`). Finally, one can add a transformation to the images.

The new object can then be used in a DataLoader to train a model.

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from torchvision import transforms

import matplotlib.pyplot as plt

In [ ]:
DATASET_PATH = 'MMHS150K/MMHS150K_text_in_image.csv'

dataset = pd.read_csv(DATASET_PATH)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, split, transform=None):
        """ __init__ function for CustomDataset

        Args:
            csv_file (str): Path to the csv file containing the dataset information
            img_dir (str): Path to the directory containing the images
            split (str): 'train' or 'test' or 'val'
            transform (callable, optional): Optional transform to be applied on a sample
        """
        self.dataset = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.split = split
        self.transform = transform
        
        # Number of images in the dataset
        self.dataset_len = len(self.dataset)
        
    def __len__(self):
        return self.dataset_len
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        image_index = self.dataset['index'].values[idx]
        
        img_path = self.img_dir + str(image_index) + '.jpg'
        image = Image.open(img_path)
        
        if self.transform:
            image = self.transform(image)
        
        label = self.dataset[self.dataset['index'] == image_index]['binary_hate'].values[0]
        tweet_text = self.dataset[self.dataset['index'] == image_index]['tweet_text_clean'].values[0]
        img_text = self.dataset[self.dataset['index'] == image_index]['img_text'].values[0]
        
        sample = {'image': image, 'label': label, 'tweet_text': tweet_text, 'img_text': img_text}
        
        return sample

# Example

Use the CSV created from `reformat_data.ipynb` and the images from the `MMHS150K/img_resized/` folder to create a custom dataset.
Then just use the DataLoader to plot some images, could be instead used to train a model.

In [ ]:
# Create a transform for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(20),
    transforms.ToTensor()
])

# Create a CustomDataset object
dataset = CustomDataset(csv_file=DATASET_PATH, img_dir='MMHS150K/img_resized/', split='train', transform=transform)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Plot the first 4 images
fig, axs = plt.subplots(4, 1, figsize=(40, 10))
for i, data in enumerate(dataloader):
    image = data['image']
    label = data['label']
    tweet_text = data['tweet_text']
    img_text = data['img_text']
    
    axs[i].imshow(image[0].permute(1, 2, 0))
    axs[i].set_title(f'Label: {"Hate" if label[0] == 1 else "No hate"}\nTweet Text: {tweet_text[0]}\nImage Text: {img_text[0]}')
    axs[i].axis('off')
    
    if i == 3:
        break
    
plt.tight_layout()
plt.show()